## Importando SQL e Criando o Cursor para Execução de Queries: Adicione dbname e password conforme o seu Banco de Dados

In [1]:
import psycopg2 
from psycopg2 import sql
conn = psycopg2.connect(dbname = "", user = "postgres", password = "", host = "localhost", port = "5432")
cursor = conn.cursor()

## Criando as tabelas no Banco de Dados

In [ ]:
sql_criacao = '''
DROP TABLE IF EXISTS unidade_de_conservacao CASCADE;
DROP TABLE IF EXISTS categoria_manejo CASCADE;
DROP TABLE IF EXISTS bioma CASCADE;
DROP TABLE IF EXISTS tipo_de_lixo CASCADE;
DROP TABLE IF EXISTS local_de_coleta CASCADE;
DROP TABLE IF EXISTS instituicao_organizadora CASCADE;
DROP TABLE IF EXISTS municipio CASCADE;
DROP TABLE IF EXISTS esfera_administrativa CASCADE;
DROP TABLE IF EXISTS mutirao_de_limpeza CASCADE;
DROP TABLE IF EXISTS coleta_de_lixo CASCADE;


CREATE TABLE IF NOT EXISTS categoria_manejo (
    id_manejo INT PRIMARY KEY,
    categoria_de_manejo TEXT NOT NULL,
    categoria_iucn TEXT NOT NULL,
    plano_de_manejo TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS bioma (
    id_bioma INT PRIMARY KEY,
    bioma_declarado TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS municipio (
    id_municipio INT PRIMARY KEY,
    cidade TEXT NOT NULL,
    estado TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS esfera_administrativa (
    id_esfera INT PRIMARY KEY,
    esfera_administrativa TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS unidade_de_conservacao (
    id_uc INT PRIMARY KEY,
    nome_da_uc TEXT NOT NULL,
    ano_de_criacao INT,
    area_soma_biomas FLOAT,
    area_soma_biomas_continentais FLOAT,
    area_marinha FLOAT,
    id_bioma INT,
    id_municipio INT,
    id_manejo INT,
    id_esfera INT,
    FOREIGN KEY (id_manejo) REFERENCES categoria_manejo(id_manejo),
    FOREIGN KEY (id_bioma) REFERENCES bioma(id_bioma),
    FOREIGN KEY (id_municipio) REFERENCES municipio(id_municipio),
    FOREIGN KEY (id_esfera) REFERENCES esfera_administrativa(id_esfera)
);

CREATE TABLE IF NOT EXISTS instituicao_organizadora (
    id_instituicao_organizadora INT PRIMARY KEY,
    instituicao_organizadora TEXT NOT NULL,
    carater_da_instituicao TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS mutirao_de_limpeza (
    id_mutirao INT PRIMARY KEY,
    data_do_mutirao DATE NOT NULL,
    horario_de_inicio TIME NOT NULL,
    numero_aproximado_de_participantes INT,
    duracao_total TIME NOT NULL,
    id_municipio INT,
    id_instituicao_organizadora INT,
    FOREIGN KEY (id_municipio) REFERENCES municipio(id_municipio),
    FOREIGN KEY (id_instituicao_organizadora) REFERENCES instituicao_organizadora(id_instituicao_organizadora)
);

CREATE TABLE IF NOT EXISTS local_de_coleta (
    id_mutirao INT PRIMARY KEY,
    nome_do_local TEXT NOT NULL,
    FOREIGN KEY (id_mutirao) REFERENCES mutirao_de_limpeza(id_mutirao)
);

CREATE TABLE IF NOT EXISTS coleta_de_lixo (
    id_mutirao INT PRIMARY KEY,
    plastico INT,
    borracha INT,
    madeira INT,
    produtos_texteis INT,
    papel INT,
    metal INT,
    vidro INT,
    materiais_de_pesca INT,
    outros_residuos INT,
    FOREIGN KEY (id_mutirao) REFERENCES mutirao_de_limpeza(id_mutirao)
);

CREATE TABLE IF NOT EXISTS tipo_de_lixo (
    id_mutirao INT PRIMARY KEY,
    fragmentos_de_plastico INT,
    canudos INT,
    embalagens_de_alimento INT,
    garrafas_pet INT,
    sacolas INT, 
    pneus INT,
    preservativos INT,
    embalagens_de_cigarro INT,
    jornais_panfletos_revistas_livros INT,
    latas INT, entulhos INT, animais_mortos INT,
    FOREIGN KEY (id_mutirao) REFERENCES coleta_de_lixo(id_mutirao)
);
'''

cursor.execute(sql_criacao)


## Populando o Banco de Dados com os Dataframes

In [ ]:
for indice, linha in df_municipio.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO municipio (id_municipio, cidade, estado)
            VALUES (%s, %s, %s)
        """),
        (linha['id_municipio'], linha['Cidade'], linha['Estado'])
    )

for indice, linha in df_instituicao_organizadora.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO instituicao_organizadora (id_instituicao_organizadora, instituicao_organizadora, carater_da_instituicao)
            VALUES (%s, %s, %s)
        """),
        (linha['id_instituicao_organizadora'], linha['Instituição Organizadora'], linha['Caráter da instituição'])
    )

for indice, linha in df_mutirao.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO mutirao_de_limpeza (id_mutirao, data_do_mutirao, horario_de_inicio, numero_aproximado_de_participantes,
            duracao_total, id_municipio, id_instituicao_organizadora)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """),  
        (linha['id_mutirao'], linha['Data do mutirão'], linha['Horário de início'], linha['Número aproximado de participantes (total)'],
         linha['Duração total (coleta + triagem)'],linha['id_municipio'],linha['id_instituicao_organizadora'])
    )

for indice, linha in df_local_coleta.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO local_de_coleta (id_mutirao, nome_do_local)
            VALUES (%s, %s)
        """),
        (linha['id_local_coleta'], linha['Nome do local (exemplo: praia de boa viagem)'])
    )

for indice, linha in df_esfera.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO esfera_administrativa (id_esfera, esfera_administrativa)
            VALUES (%s, %s)
        """),
        (linha['id_esfera'], linha['Esfera Administrativa'])
    )

for indice, linha in df_coleta_lixo.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO coleta_de_lixo (id_mutirao, plastico, borracha,
            madeira, produtos_texteis, papel, metal, vidro, materiais_de_pesca, outros_residuos)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """),
(int(linha['id_coleta_lixo']), int(linha['Plástico (total)']), int(linha['Borracha (total)']), int(linha['Madeira (total)']),
 int(linha['Produtos têxteis (total)']), int(linha['Papel (total)']), int(linha['Metal (total)']), int(linha['Vidro (total)']),
 int(linha['Materiais de Pesca (total)']), int(linha['Outros Resíduos (total)']))
    )

for indice, linha in df_manejo.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO categoria_manejo (id_manejo, categoria_de_manejo, categoria_iucn, plano_de_manejo)
            VALUES (%s, %s, %s, %s)
        """),
        (linha['id_manejo'], linha['Categoria de Manejo'], linha['Categoria IUCN'], linha['Plano de Manejo'])
    )

for indice, linha in df_bioma.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO bioma (id_bioma, bioma_declarado)
            VALUES (%s, %s)
        """),
        (linha['id_bioma'], linha['Bioma declarado'])
    )

for indice, linha in df_uc.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO unidade_de_conservacao (id_uc, nome_da_uc, ano_de_criacao, area_soma_biomas, area_soma_biomas_continentais, area_marinha,
            id_bioma, id_municipio, id_manejo, id_esfera)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """),
        (linha['id_uc'], linha['Nome da UC'], linha['Ano de Criacao'], linha['Area soma biomas'], linha['Area soma biomas continentais'], linha['Area Marinha'], 
         linha['id_bioma'], linha['id_municipio'], linha['id_manejo'], linha['id_esfera'])
    )

for indice, linha in df_dados_coleta.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO tipo_de_lixo (id_mutirao, fragmentos_de_plastico, canudos, embalagens_de_alimento, garrafas_pet, sacolas, pneus,
            preservativos, embalagens_de_cigarro, jornais_panfletos_revistas_livros, latas, entulhos, animais_mortos)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """),
(int(linha['id_dados_coleta']), int(linha['Fragmentos de plástico']), int(linha['Canudos']), int(linha['Embalagens de alimento']),
 int(linha['Garrafas PET']), int(linha['Sacolas']), int(linha['Pneus']), int(linha['Preservativos']), int(linha['Embalagens de cigarro']),
 int(linha['Jornais/panfletos/revistas/livros']), int(linha['Latas']), int(linha['Entulhos']), int(linha['Animais Mortos']))
    )

conn.commit()

## Query 1  -  Proporção de Unidades de Conservação vs. Lixo Extraído por Cidade em Mutirões de Limpeza

Esta query analisa a relação entre a presença de unidades de conservação ambiental e a quantidade de lixo coletado em mutirões de limpeza por cidade. O objetivo é entender se áreas com mais proteção ambiental tendem a gerar menos resíduos ou se há outras correlações relevantes.

In [ ]:
query_1 = '''
    SELECT
    	cidade, COUNT(nome_da_uc) AS qtd_uc, SUM(lixo_total) AS lixo_total_municipio
    FROM (
    	SELECT
    		"municipio".cidade,
    		"unidade_de_conservacao".nome_da_uc,
    		"mutirao_de_limpeza".id_municipio,
    		"mutirao_de_limpeza".id_mutirao,
    		(plastico + borracha + madeira + produtos_texteis + papel + 
    		 metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total
    	FROM
    		"coleta_de_lixo"
    	INNER JOIN "mutirao_de_limpeza" ON "coleta_de_lixo".id_mutirao = "mutirao_de_limpeza".id_mutirao
    	LEFT JOIN "unidade_de_conservacao" ON "mutirao_de_limpeza".id_municipio = "unidade_de_conservacao".id_municipio
    	INNER JOIN "municipio" ON "mutirao_de_limpeza".id_municipio = "municipio".id_municipio
    ) AS new_query
    GROUP BY cidade
    ORDER BY qtd_uc
'''
cursor.execute(query_1)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_1 = pd.DataFrame(resultado, columns=colunas)

## Query 2 - Proporção de Lixo Extraído vs. Unidades de Conservação por Estado

Esta query compara a quantidade de lixo coletado em mutirões de limpeza com o número de unidades de conservação em cada estado. O objetivo é identificar se estados com mais áreas protegidas têm menor volume de resíduos ou se há padrões distintos na geração de lixo.

In [ ]:
query_2 = '''
    SELECT
    	estado, COUNT(nome_da_uc) AS qtd_uc, SUM(lixo_total) AS lixo_total_estado
    FROM (
    	SELECT
    		"coleta_de_lixo".id_mutirao,
    		"municipio".estado,
    		"unidade_de_conservacao".nome_da_uc,
    		"mutirao_de_limpeza".id_municipio,
    		"mutirao_de_limpeza".id_mutirao,
    		(plastico + borracha + madeira + produtos_texteis + papel + 
    		 metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total
    	FROM
    		"coleta_de_lixo"
    	INNER JOIN "mutirao_de_limpeza" ON "coleta_de_lixo".id_mutirao = "mutirao_de_limpeza".id_mutirao
    	LEFT JOIN "unidade_de_conservacao" ON "mutirao_de_limpeza".id_municipio = "unidade_de_conservacao".id_municipio
    	INNER JOIN "municipio" ON "mutirao_de_limpeza".id_municipio = "municipio".id_municipio
    ) AS new_query
    GROUP BY estado
    ORDER BY lixo_total_estado;
'''
cursor.execute(query_2)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_2 = pd.DataFrame(resultado, columns=colunas)

## Query 3 - Análise de Resíduos por Bioma em Mutirões de Limpeza

Esta query avalia a distribuição de tipos e quantidades de lixo coletado em mutirões de limpeza, segmentado por biomas brasileiros. O objetivo é identificar padrões de descarte e potencial impacto ambiental em diferentes ecossistemas.

In [ ]:
query_3 = '''
    SELECT (plastico_total + canudo_total + embalagens_de_alimento_total + garrafas_pet_total + sacolas_total + pneus_total +
	   preservativos_total + embalagens_de_cigarro_total + jornais_panfletos_revistas_livros_total +
	   latas_total + entulhos_total) AS qtd_lixo_total, *
    FROM(
    	SELECT b.bioma_declarado, COUNT(mcp.cidade) AS cidades_total, SUM(fragmentos_de_plastico) AS plastico_total, SUM(canudos) AS canudo_total,
    	       SUM(embalagens_de_alimento) AS embalagens_de_alimento_total, SUM(garrafas_pet) AS garrafas_pet_total,
    		   SUM(sacolas) AS sacolas_total, SUM(pneus) AS pneus_total, SUM(preservativos) AS preservativos_total, SUM(embalagens_de_cigarro) AS embalagens_de_cigarro_total,
    		   SUM(jornais_panfletos_revistas_livros) AS jornais_panfletos_revistas_livros_total, SUM(latas) AS latas_total, SUM(entulhos) AS entulhos_total
    	FROM "tipo_de_lixo" AS t
    	JOIN "mutirao_de_limpeza" AS m ON m.id_mutirao = t.id_mutirao
    	JOIN "unidade_de_conservacao" AS u ON u.id_municipio = m.id_municipio
    	JOIN "municipio" AS mcp ON mcp.id_municipio = m.id_municipio
    	JOIN "bioma" AS b ON b.id_bioma = u.id_bioma
    	GROUP BY bioma_declarado
    )
ORDER BY qtd_lixo_total
'''
cursor.execute(query_3)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_3 = pd.DataFrame(resultado, columns=colunas)

## Query 4 - Eficiência das Instituições Organizadoras por Natureza (Público vs. Privado)

Esta query avalia o desempenho de instituições organizadoras de mutirões de limpeza, categorizadas por seu caráter (público, privado, ONG, etc.), com base no volume total de resíduos coletados. O objetivo é identificar quais tipos de organizações têm maior impacto nas ações de limpeza.

In [ ]:
query_4 = '''
    SELECT carater_da_instituicao, COUNT(id_mutirao) AS qtd_mutiroes, SUM(lixo_total) AS lixo_total
    FROM(
    	SELECT
    		"instituicao_organizadora".carater_da_instituicao,
    		"municipio".cidade,
    		"mutirao_de_limpeza".id_municipio,
    		"mutirao_de_limpeza".id_mutirao,
    		(plastico + borracha + madeira + produtos_texteis + papel + 
    		 metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total
    	FROM
    		"coleta_de_lixo"
    	INNER JOIN "mutirao_de_limpeza" ON "coleta_de_lixo".id_mutirao = "mutirao_de_limpeza".id_mutirao
    	INNER JOIN "municipio" ON "mutirao_de_limpeza".id_municipio = "municipio".id_municipio
    	INNER JOIN "instituicao_organizadora" ON 
    		"instituicao_organizadora".id_instituicao_organizadora = "mutirao_de_limpeza".id_instituicao_organizadora
    )
    GROUP BY carater_da_instituicao
    ORDER BY lixo_total;
'''
cursor.execute(query_4)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_4 = pd.DataFrame(resultado, columns=colunas)

## Query 5 - Análise de Eficiência dos Mutirões de Limpeza por Ano

Esta query avalia a eficiência dos mutirões de limpeza em um ano específico, com métricas como:

- Volume total de resíduos coletados

- Produtividade por participante

- Intensidade da coleta (lixo por minuto)

- Comparativo entre meses/estações do ano

In [ ]:
query_5 = '''
    SELECT *, (lixo_total_ano/participantes_no_ano) AS lixo_extraido_por_pessoa, (lixo_total_ano/(EXTRACT(EPOCH FROM tempo_total_de_mutirao::INTERVAL)/60))
           AS lixo_extraido_por_minuto
    FROM(
    	SELECT EXTRACT(YEAR FROM data_do_mutirao) AS ano, SUM(numero_aproximado_de_participantes) AS participantes_no_ano, 
    		   SUM(duracao_total::INTERVAL) AS tempo_total_de_mutirao, COUNT(id_municipio) AS total_municipios, SUM(plastico) AS plastico_total,
    		   SUM(borracha) AS borracha_total, SUM(madeira) AS madeira_total, SUM(produtos_texteis) AS produtos_texteis_total,
    		   SUM(papel) AS papel_total, SUM(metal) AS metal, SUM(vidro) AS vidro_total, SUM(materiais_de_pesca) AS
    	       materiais_de_pesca_total, SUM(outros_residuos) AS outros_residuos_total, SUM(plastico + borracha + madeira + produtos_texteis
    		   + papel + metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total_ano
    	FROM "mutirao_de_limpeza" AS m
    	JOIN coleta_de_lixo AS c ON c.id_mutirao = m.id_mutirao 
    	GROUP BY ano
    	ORDER BY ano
    )
'''
cursor.execute(query_5)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_5 = pd.DataFrame(resultado, columns=colunas)